In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import json
import pymysql

### 한국은행 OpenAPI 조회 서비스 사용 
http://ecos.bok.or.kr/jsp/openapi/OpenApiController.jsp?t=main

In [2]:
"""
한국은행 openAPI를 활용하여
소비자동향조사 받아옴
"""
def getEcosAPI(API_KEY, code, max, start_month, end_month):
    url = 'http://ecos.bok.or.kr/api/StatisticSearch/%s/json/kr/1/%s/%s/MM/%s/%s/?/?/?/' % (API_KEY, max, code, start_month, end_month)
    result = urlopen(url)
    html = result.read()
    
    return json.loads(html)


In [3]:
API_KEY = "YOUR_API_KEY"

data = getEcosAPI(API_KEY, code = '040Y002', max = 100000, start_month = '201801', end_month = '202010')

### 소비자심리지수 구하기
ITEM_CODE1 = 'FME'

In [4]:
data = data["StatisticSearch"]["row"]
produce = pd.DataFrame(data)

In [5]:
ccsi = produce[produce['ITEM_CODE1'] == 'FME']
ccsi

,UNIT_NAME,STAT_NAME,ITEM_CODE1,STAT_CODE,ITEM_CODE2,ITEM_CODE3,ITEM_NAME1,ITEM_NAME2,DATA_VALUE,ITEM_NAME3,TIME
17424,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,110.3,,201801
17425,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,108.4,,201802
17426,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,108.3,,201803
17427,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,107.3,,201804
17428,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,108.2,,201805
17429,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,105.6,,201806
17430,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,100.9,,201807
17431,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,99,,201808
17432,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,100.1,,201809
17433,,"9.2.1 소비자동향조사(한국은행, 전국)(월)(2008년9월~)",FME,040Y002,99988,,소비자심리지수,전체,99.3,,201810


In [6]:
ccsi = ccsi.loc[:, ['TIME', 'DATA_VALUE']].reset_index(drop=True)
ccsi

,TIME,DATA_VALUE
0,201801,110.3
1,201802,108.4
2,201803,108.3
3,201804,107.3
4,201805,108.2
5,201806,105.6
6,201807,100.9
7,201808,99
8,201809,100.1
9,201810,99.3


### DB연동

In [15]:
conn = pymysql.connect(host = "127.0.0.1", user = [USER], passwd = [PASSWORD], db = [DATABASE])
cur = conn.cursor()

### DB table 생성

In [16]:
"""
CCSI 저장할 table 생성
"""

create_table_query = """
CREATE TABLE IF NOT EXISTS ccsi(
    id BIGINT(7) NOT NULL AUTO_INCREMENT,
    year bigint(4) NOT NULL,
    month bigint(2) NOT NULL,
    ccsi double,
    primary key(id) )
    charset=utf8mb4;
"""
cur.execute(create_table_query)
# charset=utf8mb4; 유니코드 스트링 저장

0

In [17]:
cur.execute("desc ccsi")
table = cur.fetchall()
pd.DataFrame(list(table))

,0,1,2,3,4,5
0,id,bigint(7),NO,PRI,None,auto_increment
1,year,bigint(4),NO,,None,
2,month,bigint(2),NO,,None,
3,ccsi,double,YES,,None,


### CCSI 월별로 저장

In [18]:
"""
날짜에서 year, month 추출
"""
def extract_year_and_month(period):
    year = period[0:4]
    month = period[4:6]
    return year, month


In [19]:
"""
db table에 저장
"""
for i in range(len(ccsi)):
    query = """ 
    Insert into ccsi (year, month, ccsi) values (%d, %d, %f) ;
    """
    year, month = extract_year_and_month(ccsi.iloc[i].TIME)
    ccsi_value = ccsi.iloc[i].DATA_VALUE

    mystring = (query % (int(year), int(month), float(ccsi_value)))
    print(mystring)
    cur.execute(mystring)

 
    Insert into ccsi (year, month, ccsi) values (2018, 1, 110.300000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2018, 2, 108.400000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2018, 3, 108.300000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2018, 4, 107.300000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2018, 5, 108.200000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2018, 6, 105.600000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2018, 7, 100.900000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2018, 8, 99.000000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2018, 9, 100.100000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2018, 10, 99.300000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2018, 11, 95.700000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (2018, 12, 96.900000) ;
    
 
    Insert into ccsi (year, month, ccsi) values (20

In [20]:
# 확인
query = """ 
select * from ccsi ;
"""
cur.execute(query)
tmp = pd.DataFrame(list(cur.fetchall()))
tmp

,0,1,2,3
0,1,2018,1,110.3
1,2,2018,2,108.4
2,3,2018,3,108.3
3,4,2018,4,107.3
4,5,2018,5,108.2
5,6,2018,6,105.6
6,7,2018,7,100.9
7,8,2018,8,99.0
8,9,2018,9,100.1
9,10,2018,10,99.3


In [21]:
conn.commit()
cur.close()
conn.close()